In [1]:
import json
import numpy as np
import pandas as pd
from collections import defaultdict

import plotly.io as pio
import plotly.express as px
import matplotlib.pyplot as plt

pio.renderers.default = 'browser'
plt.rcParams["figure.figsize"] = (20,7)

In [2]:
us_abbrev_to_state = {"FE": "Federal","AL":"Alabama","AK":"Alaska","AZ":"Arizona","AR":"Arkansas","CA":"California","CO":"Colorado","CT":"Connecticut","DE":"Delaware","FL":"Florida","GA":"Georgia","HI":"Hawaii","ID":"Idaho","IL":"Illinois","IN":"Indiana","IA":"Iowa","KS":"Kansas","KY":"Kentucky","LA":"Louisiana","ME":"Maine","MD":"Maryland","MA":"Massachusetts","MI":"Michigan","MN":"Minnesota","MS":"Mississippi","MO":"Missouri","MT":"Montana","NE":"Nebraska","NV":"Nevada","NH":"New Hampshire","NJ":"New Jersey","NM":"New Mexico","NY":"New York","NC":"North Carolina","ND":"North Dakota","OH":"Ohio","OK":"Oklahoma","OR":"Oregon","PA":"Pennsylvania","RI":"Rhode Island","SC":"South Carolina","SD":"South Dakota","TN":"Tennessee","TX":"Texas","UT":"Utah","VT":"Vermont","VA":"Virginia","WA":"Washington","WV":"West Virginia","WI":"Wisconsin","WY":"Wyoming"}
us_state_to_abbrev = {v: k for k, v in us_abbrev_to_state.items()}

In [7]:
data = pd.read_csv("../data/executions.csv")
data["State Code"] = data["State"].apply(lambda x: us_state_to_abbrev[x])

In [8]:
data

,Unnamed: 0,Year,First Name,Last Name,Race,Sex,Region,State,County,Number of Victims,...,Other Female Victims,White Convicted,Other Convicted,Male Convicted,Female Convicted,White Victims,Other Victims,Male Victims,Female Victims,State Code
0,0,1977,Gary,Gilmore,White,Male,West,Utah,Utah County,1,...,0,1,0,1,0,1,0,1,0,UT
1,1,1979,John,Spenkelink,White,Male,South,Florida,Leon County,1,...,0,1,0,1,0,1,0,1,0,FL
2,2,1979,Jesse,Bishop,White,Male,West,Nevada,Clark County,1,...,0,1,0,1,0,1,0,1,0,NV
3,3,1981,Steven,Judy,White,Male,Midwest,Indiana,Morgan County,4,...,0,1,0,1,0,4,0,2,2,IN
4,4,1982,Frank,Coppola,White,Male,South,Virginia,Newport News city,1,...,0,1,0,1,0,1,0,1,0,VA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1522,1522,2020,Lezmond,Mitchell,Native American,Male,Federal,Federal,Federal,2,...,2,0,1,1,0,0,2,0,2,FE
1523,1523,2020,Keith,Nelson,White,Male,Federal,Federal,Federal,1,...,0,1,0,1,0,1,0,0,1,FE
1524,1524,2020,William,LeCroy,White,Male,Federal,Federal,Federal,1,...,0,1,0,1,0,1,0,0,1,FE
1525,1525,2020,Christopher,Vialva,Black,Male,Federal,Federal,Federal,2,...,0,0,1,1,0,2,0,1,1,FE


## BY STATE

In [9]:
executions_by_state = pd.concat ([data.groupby("State").sum()[["White Convicted", "Other Convicted"]], 
                                 data.groupby("State").count()["Race"]], axis=1)
executions_by_state = executions_by_state.reset_index()
executions_by_state["State Code"] = executions_by_state["State"].apply(lambda x: us_state_to_abbrev[x])
executions_by_state = executions_by_state.rename(columns={"Race": "Executions"})
executions_by_state["Executions Scaled"] = np.log(executions_by_state["Executions"])+1

#Tilføje ratio eller lige meget, når vi alligevel viser count?
#Giver desuden problemer hvis eks. 1 hvid og 0 other (1/0)
executions_by_state["White/Other ratio"] = np.round(executions_by_state["White Convicted"] / executions_by_state["Other Convicted"], 2)

In [10]:
states = set(list(executions_by_state["State Code"]))
for abbrev in list(us_abbrev_to_state.keys()):
    new_row = {}
    if abbrev not in states:
        state = us_abbrev_to_state[abbrev]
        new_row = {"State": state, "White Convicted": 0, "Other Convicted": 0, "Executions": 0, "State Code": abbrev, 
                   "Executions Scaled": 0, "White/Other ratio": "No executions"}
        executions_by_state = executions_by_state.append(new_row, ignore_index=True)

In [11]:
fig = px.choropleth(
    data_frame=executions_by_state,
    locationmode="USA-states",
    locations="State Code",
    color="Executions Scaled",
    color_continuous_scale=px.colors.sequential.YlOrRd,
    range_color=[0,8],
    hover_name="State",
    hover_data={"Executions": True, "White Convicted": True, "Other Convicted": True, "State Code": False, 
                "Executions Scaled": False, "White/Other ratio": False},
    title="Death Row Executions 1977-2016",
    scope="usa",
    template="plotly_dark"
)
#fig.update_geos(visible=False)
fig.show()

In [14]:
executions_by_state

,State,White Convicted,Other Convicted,Executions,State Code,Executions Scaled,White/Other ratio
0,Alabama,38,29,67,AL,5.204693,1.31
1,Arizona,29,8,37,AZ,4.610918,3.62
2,Arkansas,20,11,31,AR,4.433987,1.82
3,California,8,5,13,CA,3.564949,1.6
4,Colorado,1,0,1,CO,1.000000,inf
5,Connecticut,1,0,1,CT,1.000000,inf
6,Delaware,8,8,16,DE,3.772589,1
7,Federal,6,5,11,FE,3.397895,1.2
8,Florida,62,37,99,FL,5.595120,1.68
9,Georgia,48,28,76,GA,5.330733,1.71


#### Specific State

Maybe an option to choose a specific state that shows only a map and other chart data for this state?

In [15]:
state = "Alaska"

df = executions_by_state[executions_by_state["State"]==state]

fig = px.choropleth(
    data_frame=df,
    locationmode="USA-states",
    locations="State Code",
    color="Executions Scaled",
    color_continuous_scale=px.colors.sequential.YlOrRd,
    range_color=[0,8],
    hover_name="State",
    hover_data={"Executions": True, "White Convicted": True, "Other Convicted": True, "State Code": False, 
                "Executions Scaled": False},
    title=f"Death Row Executions in {state} between 1977-2016",
    scope="usa",
    template="plotly_dark",
)
#fig.update_geos(visible=False)
fig.show()

## BY YEAR AND STATE

In [24]:
executions_by_year = pd.concat ([data.groupby(["Year", "State Code"]).sum()[["White Convicted", "Other Convicted"]], 
                                 data.groupby(["Year", "State Code"]).count()["Race"]], axis=1)
executions_by_year = executions_by_year.reset_index()
executions_by_year = executions_by_year.rename(columns={"Race": "Executions"})
executions_by_year["Executions Scaled"] = np.log(executions_by_year["Executions"]) + 1

In [25]:
years = set(list(data["Year"]))
for year in years:
    states = list(executions_by_year[executions_by_year["Year"]==year]["State Code"])
    for abbrev in list(us_abbrev_to_state.keys()):
        new_row = {}
        if abbrev not in states:
            new_row = {"Year": year, "State Code": abbrev, "White Convicted": 0, "Other Convicted": 0, "Executions": 0,
                      "Executions Scaled": 0}
            executions_by_year = executions_by_year.append(new_row, ignore_index=True)

executions_by_year["State"] = executions_by_year["State Code"].apply(lambda x: us_abbrev_to_state[x])

In [35]:
executions_by_year[(executions_by_year['Year'].isin([2010, 2011])) & (executions_by_year['State']=='Alabama')]

,Year,State Code,White Convicted,Other Convicted,Executions,Executions Scaled,State
326,2010,AL,4,1,5,2.609438,Alabama
338,2011,AL,3,3,6,2.791759,Alabama


In [54]:
test

,Year,State Code,White Convicted,Other Convicted,Executions,Executions Scaled,State
326,2010,AL,4,1,5,2.609438,Alabama
327,2010,AZ,0,1,1,1.000000,Arizona
328,2010,FL,1,0,1,1.000000,Florida
329,2010,GA,2,0,2,1.693147,Georgia
330,2010,LA,1,0,1,1.000000,Louisiana
...,...,...,...,...,...,...,...
1743,2011,VT,0,0,0,0.000000,Vermont
1744,2011,WA,0,0,0,0.000000,Washington
1745,2011,WV,0,0,0,0.000000,West Virginia
1746,2011,WI,0,0,0,0.000000,Wisconsin


In [66]:
#executions_by_year.to_csv('../data/timeline_map_data.csv')
test = executions_by_year[executions_by_year['Year'].isin(['2010', '2011'])]
test = test.groupby(['State', 'State Code']).sum().reset_index()
test['Executions Scaled'] = np.log(test["Executions"]+1)

In [67]:
test

,State,State Code,Year,White Convicted,Other Convicted,Executions,Executions Scaled
0,Alabama,AL,4021,7,4,11,2.484907
1,Alaska,AK,4021,0,0,0,0.000000
2,Arizona,AZ,4021,3,2,5,1.791759
3,Arkansas,AR,4021,0,0,0,0.000000
4,California,CA,4021,0,0,0,0.000000
5,Colorado,CO,4021,0,0,0,0.000000
6,Connecticut,CT,4021,0,0,0,0.000000
7,Delaware,DE,4021,1,0,1,0.693147
8,Federal,FE,4021,0,0,0,0.000000
9,Florida,FL,4021,2,1,3,1.386294


In [12]:
year = 1999 # Between 1977-2020

df = executions_by_year[executions_by_year["Year"]==year]

fig = px.choropleth(
    data_frame=df,
    locationmode="USA-states",
    locations="State Code",
    color="Executions Scaled",
    color_continuous_scale=px.colors.sequential.YlOrRd,
    range_color=[0,5],
    hover_name="State",
    hover_data={"Executions": True, "White Convicted": True, "Other Convicted": True, "State Code": False, 
                "Executions Scaled": False},
    title=f"Death Row Executions in year {year}",
    scope="usa",
    template="plotly_dark",
)
#fig.update_geos(visible=False)
fig.show()

Side by side maps for every year

In [13]:
year = 1999 # Between 1977-2020

df = executions_by_year[executions_by_year["Year"]==year]

fig = px.choropleth(
    data_frame=executions_by_year,
    locationmode="USA-states",
    locations="State Code",
    color="Executions Scaled",
    color_continuous_scale=px.colors.sequential.YlOrRd,
    range_color=[0,5],
    hover_name="State",
    hover_data={"Executions": True, "White Convicted": True, "Other Convicted": True, "State Code": False, 
                "Executions Scaled": False},
    title=f"Death Row Executions per year",
    scope="usa",
    template="plotly_dark",
    facet_col="Year",
    facet_col_wrap=9
)
#fig.update_geos(visible=False)
fig.show()

#### Specific state

In [14]:
year = 1999 # Between 1977-2020
state = "Texas"

df = executions_by_year[executions_by_year["Year"]==year]
df = df[df["State"]==state]

fig = px.choropleth(
    data_frame=df,
    locationmode="USA-states",
    locations="State Code",
    color="Executions Scaled",
    color_continuous_scale=px.colors.sequential.YlOrRd,
    range_color=[0,5],
    hover_name="State",
    hover_data={"Executions": True, "White Convicted": True, "Other Convicted": True, "State Code": False, 
                "Executions Scaled": False},
    title=f"Death Row Executions in {state}, year {year}",
    scope="usa",
    template="plotly_dark"
)
#fig.update_geos(visible=False)
fig.show()